In [1]:
import pandas as pd

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch

import numpy as np
from sklearn.metrics import classification_report

C:\Users\assas\AppData\Local\Temp\ipykernel_764\3296178993.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
c:\Users\assas\anaconda3\envs\ML_development\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.is_available()

True

In [3]:
LR = 2e-5
EPOCHS = 1
BATCH_SIZE = 16
MODEL = "cardiffnlp/twitter-xlm-roberta-base"
MAX_TRAINING_EXAMPLES = -1 # set this to -1 if you want to use the whole training set

In [4]:
train = pd.read_csv('../csv/tweet-tone-train.csv')
x_train = train['Tweets']
y_train = train['Label'].replace({"Kızgın": 0, "Korku": 1, "Mutlu": 2, "Sürpriz": 3, "Üzgün" : 4})
test = pd.read_csv('../csv/tweet-tone-test.csv')
x_test = test['Tweets']
y_test = test['Label'].replace({"Kızgın": 0, "Korku": 1, "Mutlu": 2, "Sürpriz": 3, "Üzgün" : 4})

C:\Users\assas\AppData\Local\Temp\ipykernel_764\3507238578.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_train = train['Label'].replace({"Kızgın": 0, "Korku": 1, "Mutlu": 2, "Sürpriz": 3, "Üzgün" : 4})
C:\Users\assas\AppData\Local\Temp\ipykernel_764\3507238578.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_test = test['Label'].replace({"Kızgın": 0, "Korku": 1, "Mutlu": 2, "Sürpriz": 3, "Üzgün" : 4})


In [5]:
tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=True)

In [6]:
x_train = x_train.apply(lambda x: tokenizer(x, truncation=True, padding='max_length', max_length=256))
x_test = x_test.apply(lambda x: tokenizer(x, truncation=True, padding="max_length", max_length=256))

In [7]:
x_train.reset_index(drop=True, inplace=True)

In [8]:
x_test.reset_index(drop=True, inplace=True)

max_tok = 0
for i in x_train:
    if len(i['input_ids']) > max_tok:
        max_tok = len(i['input_ids'])
max_tok
3549

max_tok = 0
for i in x_val:
    if len(i['input_ids']) > max_tok:
        max_tok = len(i['input_ids'])
max_tok
2241

max_tok = 0
for i in x_test:
    if len(i['input_ids']) > max_tok:
        max_tok = len(i['input_ids'])
max_tok
1331

In [9]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {"input_ids": torch.tensor(self.encodings[idx]["input_ids"])}
        item['attention_mask'] = torch.tensor(self.encodings[idx]["attention_mask"])
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MyDataset(x_train, y_train)
test_dataset = MyDataset(x_test, y_test)

In [10]:
training_args = TrainingArguments(
    output_dir='./results',                   # output directory
    num_train_epochs=EPOCHS,                  # total number of training epochs
    per_device_train_batch_size=BATCH_SIZE,   # batch size per device during training
    per_device_eval_batch_size=BATCH_SIZE,    # batch size for evaluation
    warmup_steps=100,                         # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                        # strength of weight decay
    logging_dir='./logs',                     # directory for storing logs
    logging_steps=10,                         # when to print log
)

model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=5)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
trainer = Trainer(
    model=model,                              # the instantiated 🤗 Transformers model to be trained
    args=training_args,                       # training arguments, defined above
    train_dataset=train_dataset,              # training dataset
)

trainer.train()

  5%|▍         | 10/221 [00:31<07:30,  2.14s/it]

{'loss': 1.612, 'learning_rate': 5e-06, 'epoch': 0.05}


  9%|▉         | 20/221 [00:51<06:41,  2.00s/it]

{'loss': 1.5955, 'learning_rate': 1e-05, 'epoch': 0.09}


 14%|█▎        | 30/221 [01:11<06:20,  1.99s/it]

...

100%|█████████▉| 220/221 [07:30<00:01,  2.00s/it]

{'loss': 0.0372, 'learning_rate': 4.132231404958678e-07, 'epoch': 1.0}


100%|██████████| 221/221 [07:32<00:00,  2.05s/it]

{'train_runtime': 452.8187, 'train_samples_per_second': 7.809, 'train_steps_per_second': 0.488, 'train_loss': 0.6110126912484274, 'epoch': 1.0}


TrainOutput(global_step=221, training_loss=0.6110126912484274, metrics={'train_runtime': 452.8187, 'train_samples_per_second': 7.809, 'train_steps_per_second': 0.488, 'train_loss': 0.6110126912484274, 'epoch': 1.0})

In [12]:
test_preds_raw, test_labels , _ = trainer.predict(test_dataset)
test_preds = np.argmax(test_preds_raw, axis=-1)
print(classification_report(test_labels, test_preds, digits=3))

100%|██████████| 25/25 [00:03<00:00,  7.96it/s]

              precision    recall  f1-score   support

           0      0.938     0.962     0.950        79
           1      0.987     1.000     0.994        78
           2      0.988     0.988     0.988        80
           3      0.974     0.987     0.981        77
           4      0.987     0.937     0.961        79

    accuracy                          0.975       393
   macro avg      0.975     0.975     0.975       393
weighted avg      0.975     0.975     0.975       393



In [13]:
trainer.save_model("./results/best_model") # save best model